
# Lucid Software Business Intelligence Assignment

For this assignment you will be interacting with public APIs and completing tasks using the data they provide. None of the questions are analytically challenging, most of the task is getting and processing the data from the APIs

You should complete this assignmenet by yourself, but you may use any tools or resources at your disposal (e.g., internet, textbooks, spark, python, etc.). If you are using spark or python please use a ZEPL notebook. If you prefer another environment you may share your work once you’ve completed the assignment.

The assigned tasks should be ordered in increasing difficulty. Please complete as many as you can and use your own best judgement as you make decisions about how to interpret and model the data.

When sharing your zepl workbook share it with zepl user jstark instead of over email.

### Earthquake Data
https://earthquake.usgs.gov/fdsnws/event/1/

How many earthquakes of at least 5 magnitude happened in and around Tonga in May of 2017?

In [19]:
## Import the libraries necessary for the Earthquake Data and SpaceX Launch Data
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
from pandas.io.json import json_normalize
from urllib import *
import urllib.request 
import json
from sqlalchemy import create_engine
import psycopg2
from IPython.core.display import display, HTML

In [2]:
# Initialize plotly and mapbox access
plotly.tools.set_credentials_file(username='dylan.t.lorimer@gmail.com', api_key='gZQQodjxthmQnusI8HGD')
mapbox_access_token = 'pk.eyJ1IjoiZGxvcmltZXIiLCJhIjoiY2pkbHY0b25zMGN2eTMzbzQ4ODdxeWE3YyJ9.CaSl5CksBlIL6fJOKY0kwA'

In [3]:
# Read USGS earthquake data into a dataframe and filter for necessary rows and columns

# Filtering for May 2017 earthquakes near Tonga (>=5.0 magnitude) is done in the url using the arguments: starttime, endtime, minmagnitude, longitude, latitude, maxradiuskm
df = pd.read_csv("https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2017-05-01&endtime=2017-05-31&minmagnitude=5.0&longitude=-175.198&latitude=-21.179&maxradiuskm=1000")
df = df.query('type == "earthquake"')
df1 = df[['time','place','mag','latitude','longitude']]

In [4]:
# Create interactive map for May 2017 Earthquakes of Magnitude 5.0 and Above, within 1000km of Tonga
site_lat = df['latitude']
site_lon = df['longitude']

data = Data([
    Scattermapbox(
        lat=df['latitude'],
        lon=df['longitude'],
        mode='markers',
        marker=Marker(
            size=9
        ),
        text=df['place'] + '<br>Magnitude ' + (df['mag']).astype(str),
    )]
)
        
layout = Layout(
    title='May 2017 Earthquakes of Magnitude 5.0 and Above, within 1000km of Tonga',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(lat=-21, lon=-175),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='May 2017 Earthquakes of Magnitude 5.0 and Above, within 1000km of Tonga')

In [5]:
# Print data table for earthquakes in map
df1

,time,place,mag,latitude,longitude
0,2017-05-30T19:59:57.820Z,"160km SSW of Nadi, Fiji",5.0,-19.1606,176.8782
1,2017-05-26T06:27:30.540Z,"42km S of Ndoi Island, Fiji",5.0,-21.0360,-178.6770
2,2017-05-25T06:02:20.140Z,"169km SW of Vaini, Tonga",5.6,-22.3132,-176.3299
3,2017-05-22T16:41:06.730Z,"132km S of Hihifo, Tonga",5.3,-17.1282,-173.5293
4,2017-05-16T15:29:14.480Z,South of the Fiji Islands,5.1,-23.7723,-177.6057
5,2017-05-12T00:46:56.720Z,"217km NE of Raoul Island, New Zealand",5.6,-27.6765,-176.5945
6,2017-05-09T15:25:35.720Z,"132km ESE of Pangai, Tonga",5.1,-20.4035,-173.2567
7,2017-05-07T08:08:35.760Z,"119km ESE of Neiafu, Tonga",5.0,-18.8764,-172.8761
8,2017-05-02T14:07:00.240Z,South of Tonga,5.1,-24.2588,-175.7662


### SpaceX Launch Data
https://github.com/r-spacex/SpaceX-API/wiki/Past-Launches

#### In which year did SpaceX first successfully reuse a capsule?

In [6]:
# Read in SpaceX data and perform the first stage of JSON normalization
with urllib.request.urlopen("https://api.spacexdata.com/v2/launches") as url:
    data = json.loads(url.read().decode())
df_x = json_normalize(data=data)

# rename a bunch of columns for clarity
df_x.rename(columns={'launch_site.site_id':'launch_site_id',
                     'launch_site.site_name':'launch_site_name',
                     'launch_site.site_name_long':'launch_site_name_long',
                     'rocket.rocket_id':'rocket_id',
                     'rocket.rocket_name':'rocket_name',
                     'rocket.rocket_type':'rocket_type',
                     'links.mission_patch':'mission_patch',
                     'links.reddit_campaign':'reddit_campaign',
                     'links.reddit_launch':'reddit_launch',
                     'links.reddit_recovery':'reddit_recovery',
                     'links.reddit_media':'reddit_media',
                     'links.presskit':'presskit',
                     'links.article_link':'article_link',
                     'links.video_link':'video_link'
                    }, inplace= True)

In [7]:
# Print the earliest year that a capsule was reused (True: 2017)
pd.DataFrame(df_x.groupby(['reuse.capsule'], sort=False)['launch_year'].min())

,launch_year
reuse.capsule,
False,2006
True,2017


#### Which flights used core Merlin 1C?

In [8]:
# Build the cores table

# This table is a record of events associated with each core's flight, so
# cores will appear multiple times. (I chose to give cores their own table to
# answer the question quickly and so I didn't make the flights table too busy 
# later on)

# Create a blank dataframe to populate 
cores_df = pd.DataFrame(columns=['core_serial', 'flight', 'land_success',
       'landing_type', 'landing_vehicle', 'reused', 'rocket_id'])

for i in range(len(df_x)):
    row_cores = json_normalize(df_x['rocket.first_stage.cores'][i])
    row_cores['flight_number'] = df_x['flight_number'][i]
    row_cores['rocket_id'] = df_x['rocket_id'][i]
    cores_df = pd.concat([cores_df, row_cores])

# Clean up and clarify the table
cores_df.rename(columns={'flight': 'flight_count'}, inplace=True) # now flight number represents the unique identifier for the flights and flight count represents the number of imes that the core has flown
cores_df = cores_df.reset_index(drop=True)
cores_df.drop(cores_df.columns[0],axis=1,inplace=True)

In [9]:
# Print out the flights that used the 'Merlin 1C' core
pd.DataFrame(cores_df[['flight_number','core_serial']][cores_df['core_serial'] == 'Merlin 1C'])

,flight_number,core_serial
2,3.0,Merlin 1C
3,4.0,Merlin 1C
4,5.0,Merlin 1C


#### Transform the past launch data into the following relational tables and document your model


In [10]:
# Build the payloads table

# Create a blank dataframe to populate 
payloads_df = pd.DataFrame(columns=['cap_serial', 'cargo_manifest', 'customers', 'flight_number',
       'flight_time_sec', 'mass_returned_kg', 'mass_returned_lbs', 'orbit',
       'payload_id', 'payload_mass_kg', 'payload_mass_lbs', 'payload_type',
       'reused', 'rocket_id'])

for i in range(len(df_x)):
    row_payloads = json_normalize(df_x['rocket.second_stage.payloads'][i])
    row_payloads['flight_number'] = df_x['flight_number'][i]
    row_payloads['rocket_id'] = df_x['rocket_id'][i]
    payloads_df = pd.concat([payloads_df, row_payloads])

# Reorganize columns
payloads_df = payloads_df.reset_index(drop=True)
payloads_cols = payloads_df.columns.tolist()
payloads_cols = payloads_cols[8:9] + payloads_cols[3:4] + payloads_cols[-1:] + payloads_cols[0:2] + payloads_cols[4:7] + payloads_cols[9:12] + payloads_cols[7:8] + payloads_cols[2:3]
payloads_df = payloads_df[payloads_cols]

In [11]:
# Build the flights table

flights_df = df_x[['flight_number',
                  'launch_date_utc', # took out other times because it's a simple transformation
                  'launch_date_local',
                  'rocket_id',
                  'reuse.core',
                  'reuse.side_core1',
                  'reuse.side_core2',
                  'reuse.fairings',
                  'reuse.capsule',
                  'telemetry.flight_club',
                  'launch_site_id',
                  'launch_success',
                  'details']].copy()

flights_df['details'] = flights_df['details'].str[:200]

# Reorganize columns
flights_cols = flights_df.columns.tolist()
flights_cols = flights_cols[:1] + flights_cols[3:4] + flights_cols[1:3] + flights_cols[4:]
flights_df = flights_df[flights_cols]

In [12]:
# Build the launch_sites table

launch_sites_df = df_x[['launch_site_id', # moved these into their own dataframe because it was becoming an unwieldy table
                  'launch_site_name',
                  'launch_site_name_long']]


In [13]:
# Build the flight_links table

# Again, to simplify the flights table for end users, I opted to create a
# separate table for the flight links

flight_links_df = df_x[['flight_number', # moved these into their own dataframe because it was becoming an unwieldy table
                  'mission_patch',
                  'reddit_campaign',
                  'reddit_launch',
                  'reddit_recovery',
                  'reddit_media',
                  'presskit',
                  'article_link',
                  'video_link']]


In [14]:
# Build the rockets table

rockets_df = df_x.groupby(['rocket_id',
              'rocket_name',
              'rocket_type'])['launch_date_utc'].min()
rockets_df = rockets_df.reset_index()
# Rename column for clarity
rockets_df.rename(columns={'launch_date_utc':'type_effective_date'}, inplace=True)

In [15]:
# Build the customers table

# Get all customers for each row in payload_df's 'customers' column
customers_df = payloads_df.set_index(['payload_id'])['customers'].apply(pd.Series).stack()
customers_df = customers_df.reset_index()
customers_df.columns = ['payload_id', 'customer_number','customer']
customers_df.reset_index()
customers_df['pc_id'] = range(1, len(customers_df) + 1)

# Re-order the columns so they make a little more sense
cols = customers_df.columns.tolist()
cols = cols[-1:] + cols[:1] + cols[2:] + cols[1:-1]
customers_df = customers_df[cols]


# Pull the customers column out of payload_df
payloads_df = payloads_df.drop(['customers'], axis=1)

In [16]:
my_dfs = [cores_df, payloads_df, flights_df, launch_sites_df, flight_links_df, rockets_df, customers_df]
df_names = ['x_cores', 'x_payloads', 'x_flights', 'x_launch_sites', 'x_flight_links', 'x_rockets', 'x_payload_customers']

In [17]:
user_id = input("Input your Lucid Redshift user_id: ")
password = input("Input your Lucid Redshift password: ")

Input your Lucid Redshift user_id: dylan
Input your Lucid Redshift password: 3sis=RLM


In [18]:
conn = create_engine('postgresql://' + user_id + ':' + password + '@' + 'lucid-redshift-prod-enc.czguhkk4ukda.us-east-1.redshift.amazonaws.com:5439/lucid')

for df in range(len(my_dfs)):
    my_dfs[df].to_sql(df_names[df], conn, index = False, if_exists = 'replace')

In [21]:
display(HTML('<div style="width: 640px; height: 480px; margin: 10px; position: relative;"><iframe allowfullscreen frameborder="0" style="width:640px; height:480px" src="https://www.lucidchart.com/documents/embeddedchart/c5f1d96c-b41a-4000-b345-8d72130f8549" id="Vkdf1x9IlxCA"></iframe></div>'))